In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
 
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torchvision.utils import make_grid
import torchvision.models as models
import matplotlib.pyplot as plt
%matplotlib inline

from resnet9 import ResNet9
from pipeline_jovian import Pipeline_v3

#### Computing Mean and Std of CIFAR-100 Training Images


In [2]:
%%time

train_dataset = torchvision.datasets.CIFAR100(root='./data',
                                              train=True,
                                              download=True,
                                              transform=transforms.ToTensor())
# Loading all images without batches
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=None,
                                           num_workers=2,
                                           pin_memory=True,
                                           shuffle=False)

mean, std  = torch.Tensor([0,0,0]), torch.Tensor([0,0,0])

for i, (image, labels) in enumerate(train_loader):
    mean += image.mean([1,2])
    std += image.std([1,2])

stats = ((mean / len(train_dataset)).tolist(), (std / len(train_dataset)).tolist())

print(mean / len(train_dataset), std / len(train_dataset))

Files already downloaded and verified
tensor([0.5071, 0.4866, 0.4409]) tensor([0.2009, 0.1984, 0.2023])
CPU times: user 24.6 s, sys: 11.8 s, total: 36.3 s
Wall time: 31.7 s


In [3]:
train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4,padding_mode='reflect'), 
                                           transforms.RandomHorizontalFlip(), 
                                           transforms.ToTensor(), 
                                           transforms.Normalize(*stats,inplace=True)
                                          ])

test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize(*stats)])

In [4]:
model = Pipeline_v3(model = ResNet9(in_channels=3,
                                    num_classes=100),
                    loss = 'cross_entropy_loss',
                    optimizer = 'adam',
                    learning_rate = 0.01,
                    batch_size = 400,
                    num_epochs=25,
                    weight_decay = 1e-4,
                    train_transform = train_transform,
                    test_transform = test_transform)

Using device: GPU



In [5]:
%%time

model.runModel(grad_clip=0.1)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified

Train set size: 50000
Val set size: 10000
Test set size: 10000

Applied Transformations and loaded data

batch size: 400, # batches(steps per epoch): 125

Training...

Epoch 1/25
----------
train Loss: 3.6433 Acc: 0.1520
val Loss: 3.3518 Acc: 0.2154

Epoch 2/25
----------
train Loss: 2.7666 Acc: 0.2973
val Loss: 2.8494 Acc: 0.3243

Epoch 3/25
----------
train Loss: 2.4321 Acc: 0.3735
val Loss: 2.9713 Acc: 0.3274

Epoch 4/25
----------
train Loss: 2.0783 Acc: 0.4466
val Loss: 2.6989 Acc: 0.3576

Epoch 5/25
----------
train Loss: 1.8737 Acc: 0.4908
val Loss: 2.1636 Acc: 0.4377

Epoch 6/25
----------
train Loss: 1.8168 Acc: 0.5127
val Loss: 2.5080 Acc: 0.4055

Epoch 7/25
----------
train Loss: 1.6842 Acc: 0.5438
val Loss: 2.3877 Acc: 0.4437

Epoch 8/25
----------
train Loss: 1.4631 Acc: 0.5905
val Loss: 1.8591 Acc: 0.4991

Epoch 9/25
----------
train Loss: 1.3749 Acc: 0.6097
